# API Keys

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

# Map-reduce

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Prompts
subjects_prompt = """Generate a comma separated list of between 2 and 5 examples related to: {topic}"""
joke_prompt="""Generate a joke about {subject}"""
best_joke_prompt="""Below are a bunch of jokes about {topic}. Select the best one! Return the ID of the best one."""

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
from operator import add

from typing import Annotated, TypedDict

from langchain_core.pydantic_v1 import BaseModel

# Classes
class Subjects(BaseModel):
    subjects: list[str]

class BestJoke(BaseModel):
    id: int

# State
class OverallState(TypedDict):
    topic: str
    subjects: list
    jokes: Annotated[list, add]
    best_selected_joke: str

In [ ]:
def generate_topics(state: OverallState):
    prompt = subjects_prompt.format(topic=state["topic"])
    response = llm.with_structured_output(Subjects).invoke(prompt)

In [ ]:
from langgraph.constants import Send

# State
class JokeState(TypedDict):
    subject: str

class Joke(BaseModel):
    joke: str

# Nodes
def generate_joke(state: JokeState):
    prompt = joke_prompt.format(subject=state["subject"])
    response = llm.with_structures_output(Joke).invoke(prompt)
    return {"jokes": [response.joke]}

def best_joke(state: OverallState):
    jokes = "\n\n".join(state["jokes"])
    prompt = best_joke_prompt.format(topic=state["topic"], jokes=jokes)
    response = llm.with_structured_output(BestJoke).invoke(prompt)
    return {"best_selected_joke": state["jokes"][response.id]}

# Map
def continue_to_jokes(state: OverallState):
    return [Send("generate_joke", {"subject": subject}) for subject in state["subject"]]

In [ ]:
from IPython.display import Image, display

from langgraph.graph import StateGraph, START, END

# Graph
builder = StateGraph(OverallState)
builder.add_node("generate_topics", generate_topics)
builder.add_node("generate_joke", generate_joke)
builder.add_node("best_joke", best_joke)

# Logic
builder.add_edge(START)